In [1]:
import sqlite3

In [2]:
def create_connection(db):
    conn = None
    try:
        conn = sqlite3.connect(db)
        print(sqlite3.version)
    except sqlite3.Error as e:
        print(e)
    return conn

In [3]:
def facilities_less_than_1000(conn):
    cur = conn.cursor()
    query = """
    with total_revenue as (
    select f.name as facname
    , case when b.memid=0 then b.slots*f.guestcost else 0 end as guestrev
    , case when b.memid<>0 then b.slots*f.membercost else 0 end as membrev
    from Bookings b
    left join Facilities f on f.facid=b.facid
    )
    select facname,sum(guestrev+membrev)
    from total_revenue
    group by 1
    having sum(guestrev+membrev)<1000
    order by 2 desc        """
    cur.execute(query)
    rows = cur.fetchall()
    for row in rows:
        print(row)

In [4]:
def member_recommend(conn):
    cur = conn.cursor()

    query = """
        SELECT m1.surname || ' ' || m1.firstname as member
        , m2.surname || ' ' || m2.firstname as recommended_by
        FROM `Members` as m1
        left join Members as m2 on m1.recommendedby = m2.memid
        order by m1.surname,m1.firstname
        """
    cur.execute(query)
    rows = cur.fetchall()
    for row in rows:
        print(row)

In [5]:
def facility_member_use(conn):
    cur = conn.cursor()

    query = """
    SELECT  f.name, m.surname||' '||m.firstname
    , count(*)
    from bookings b
    left join facilities f on b.facid=f.facid 
    left join members m on m.memid=b.memid
    where b.memid<>0
    group by 1,2
    order by 1,3 desc
        """
    cur.execute(query)
    rows = cur.fetchall()
    for row in rows:
        print(row)

In [6]:
def facility_monthly_use(conn):
    cur = conn.cursor()

    query = """
    SELECT  f.name, strftime('%m',b.starttime) as 'Month'
    , count(*)
    from bookings b
    left join facilities f on b.facid=f.facid 
    left join members m on m.memid=b.memid
    where b.memid<>0
    group by 1,2
    order by 1,2 asc
        """
    cur.execute(query)
    rows = cur.fetchall()
    for row in rows:
        print(row)

In [7]:
def main():
    db = 'sqlite_db_pythonsqlite.db'
    
    conn = create_connection(db)
    with conn:
        print('Q10: Produce a list of facilities with a total revenue less than 1000.')
        print(facilities_less_than_1000(conn))
        print()
        print('Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order')
        print(member_recommend(conn))
        print()
        print('Q12: Find the facilities with their usage by member, but not guests')
        print(facility_member_use(conn))
        print()
        print('Q13: Find the facilities usage by month, but not guests')
        print(facility_monthly_use(conn))        

In [8]:
if __name__=='__main__':
    main()

2.6.0
Q10: Produce a list of facilities with a total revenue less than 1000.
('Pool Table', 270)
('Snooker Table', 240)
('Table Tennis', 180)
None

Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order
('Bader Florence', 'Stibbons Ponder')
('Baker Anne', 'Stibbons Ponder')
('Baker Timothy', 'Farrell Jemima')
('Boothe Tim', 'Rownam Tim')
('Butters Gerald', 'Smith Darren')
('Coplin Joan', 'Baker Timothy')
('Crumpet Erica', 'Smith Tracy')
('Dare Nancy', 'Joplette Janice')
('Farrell David', None)
('Farrell Jemima', None)
('GUEST GUEST', None)
('Genting Matthew', 'Butters Gerald')
('Hunt John', 'Purview Millicent')
('Jones David', 'Joplette Janice')
('Jones Douglas', 'Jones David')
('Joplette Janice', 'Smith Darren')
('Mackenzie Anna', 'Smith Darren')
('Owen Charles', 'Smith Darren')
('Pinker David', 'Farrell Jemima')
('Purview Millicent', 'Smith Tracy')
('Rownam Tim', None)
('Rumney Henrietta', 'Genting Matthew')
('Sarwin Ramnaresh', 'Bader Florenc